In [51]:
import pandas as pd
import numpy as np
from phik.report import plot_correlation_matrix
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, \
    classification_report, roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [52]:
df_raw = pd.read_csv('/kaggle/input/leopard-challenge-classification/train.csv')
test = pd.read_csv(r'/kaggle/input/leopard-challenge-classification/test.csv')
df_raw.shape, test.shape

((13863, 26), (5942, 25))

In [53]:
test_cv = test.drop(['oral', 'ID'], axis = 1).copy()
test_cv['tartar'] = test_cv['tartar'].apply(lambda x: 1 if x == 'Y' else 0)
merged_data = df_raw.drop(['oral', 'ID'], axis=1).copy()
merged_data['tartar'] = merged_data['tartar'].apply(lambda x: 1 if x == 'Y' else 0)

In [54]:
# Создание бинарного признака на основе BMI
merged_data['BMI'] = merged_data['weight(kg)'] / ((merged_data['height(cm)'] / 100) ** 2)
merged_data['BMI_status'] = (merged_data['BMI'] < 18.5) | (merged_data['BMI'] >= 24.9)
merged_data['BMI_status'] = merged_data['BMI_status'].astype(int)
merged_data['log_tr'] = merged_data['triglyceride'].apply(np.log)
merged_data['log_gtp'] = merged_data['Gtp'].apply(np.log)
merged_data['log_log_alt'] = np.log(np.log(merged_data['ALT']))
merged_data['BMI_log_gtp'] = merged_data['BMI'].apply(np.log)
merged_data = merged_data.drop(['triglyceride', 'Gtp', 'ALT', 'BMI'], axis=1)

test_cv['BMI'] = test_cv['weight(kg)'] / ((test_cv['height(cm)'] / 100) ** 2)
test_cv['BMI_status'] = (test_cv['BMI'] < 18.5) | (test_cv['BMI'] >= 24.9)
test_cv['BMI_status'] = test_cv['BMI_status'].astype(int)
test_cv['log_tr'] = test_cv['triglyceride'].apply(np.log)
test_cv['log_gtp'] = test_cv['Gtp'].apply(np.log)
test_cv['log_log_alt'] = np.log(np.log(test_cv['ALT']))
test_cv['BMI_log_gtp'] = test_cv['BMI'].apply(np.log)
test_cv = test_cv.drop(['triglyceride', 'Gtp', 'ALT', 'BMI'], axis=1)

In [55]:
import optuna
from sklearn.model_selection import train_test_split, StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_recall_curve

In [56]:
from sklearn.model_selection import train_test_split


X = merged_data.drop(['smoking'], axis=1) 
Y = merged_data['smoking']

X.shape, Y.shape

((13863, 25), (13863,))

In [57]:
from sklearn.preprocessing import MinMaxScaler

# Масштабирование признаков
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
test_cv = scaler.transform(test_cv)
X.shape,  test_cv.shape

((13863, 25), (5942, 25))

In [58]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# Инициализация StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11090, 25), (2773, 25), (11090,), (2773,))

In [59]:
X_train = pd.DataFrame(X_train)

In [60]:
def objective(trial):
    # Гиперпараметры для оптимизации
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.15),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
        'class_weight': 'balanced'
    }
    
    oof_preds = np.zeros(X_train.shape[0])

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        X_fold_train, y_fold_train = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_fold_val, y_fold_val = X_train.iloc[val_idx], y_train.iloc[val_idx]

        # Создание и обучение модели с ранней остановкой
        lgbm_model = LGBMClassifier(**params)
        lgbm_model.fit(
            X_fold_train, y_fold_train,
            eval_set=[(X_fold_val, y_fold_val)],
            early_stopping_rounds=70,
            verbose=False
        )

        # Сохранение предсказаний
        oof_preds[val_idx] = lgbm_model.predict_proba(X_fold_val)[:, 1]

    # Вычисление ROC AUC на out-of-fold предсказаниях
    oof_roc_auc = roc_auc_score(y_train, oof_preds)

    return oof_roc_auc

In [61]:
%%time
# Запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)

# Вывод лучших гиперпараметров
best_params = study.best_params
print(f"Best Params: {best_params}")

[I 2023-10-12 10:05:56,775] A new study created in memory with name: no-name-0dc1615d-9622-482e-939d-5e3ef79bb5ee
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0.6625695939911336, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6625695939911336
[LightGBM] [Warning] bagging_fraction is set=0.3823664555431716, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3823664555431716
[LightGBM] [Warning] lambda_l2 is set=2.4955249143227185, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.4955249143227185
[LightGBM] [Warning] feature_fraction is set=0.9817654855116916, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9817654855116916
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0.6625695939911336, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6625695939911336
[LightGBM] [Warning] bagging_fraction is set=0.3823664555431716, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3823664555431716
[LightGBM] [Warning] lambda_l2 is set=2.4955249143227185, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.4955249143227185
[LightGBM] [Warning] feature_fraction is set=0.9817654855116916, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9817654855116916
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

[LightGBM] [Warning] lambda_l1 is set=7.738675934239081, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.738675934239081
[LightGBM] [Warning] bagging_fraction is set=0.4591640846323099, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4591640846323099
[LightGBM] [Warning] lambda_l2 is set=4.654003313876887, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.654003313876887
[LightGBM] [Warning] feature_fraction is set=0.793910174614334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.793910174614334
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=7.738675934239081, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.738675934239081
[LightGBM] [Warning] bagging_fraction is set=0.4591640846323099, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4591640846323099
[LightGBM] [Warning] lambda_l2 is set=4.654003313876887, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.654003313876887
[LightGBM] [Warning] feature_fraction is set=0.793910174614334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.793910174614334
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


[I 2023-10-12 10:27:13,317] Trial 24 finished with value: 0.6948442796610169 and parameters: {'n_estimators': 1909, 'learning_rate': 0.029923259140904303, 'max_depth': 8, 'num_leaves': 80, 'min_child_samples': 45, 'feature_fraction': 0.793910174614334, 'bagging_fraction': 0.4591640846323099, 'bagging_freq': 6, 'lambda_l1': 7.738675934239081, 'lambda_l2': 4.654003313876887}. Best is trial 2 with value: 0.7065466101694915.


Best Params: {'n_estimators': 2109, 'learning_rate': 0.01912368799501412, 'max_depth': 11, 'num_leaves': 90, 'min_child_samples': 8, 'feature_fraction': 0.6359736909429626, 'bagging_fraction': 0.4265533399132706, 'bagging_freq': 2, 'lambda_l1': 3.8476260070128565, 'lambda_l2': 2.465060612958123}
CPU times: user 23min 52s, sys: 12min 42s, total: 36min 34s
Wall time: 21min 16s


In [62]:
# Обучение с лучшими гиперпараметрами
lgbm_model = LGBMClassifier(**best_params)
lgbm_model.fit(X_train, y_train)

# Предсказания на тестовой выборке
test_preds = lgbm_model.predict_proba(X_test)[:, 1]


[LightGBM] [Warning] lambda_l1 is set=3.8476260070128565, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.8476260070128565
[LightGBM] [Warning] feature_fraction is set=0.6359736909429626, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6359736909429626
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=2.465060612958123, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.465060612958123
[LightGBM] [Warning] bagging_fraction is set=0.4265533399132706, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4265533399132706


In [65]:
lgbm_model.booster_.save_model('LGBM_Optuna_V_1_best_model_no_exta_features.txt')

# Загрузка модели
# import lightgbm as lgb

# loaded_model = lgb.Booster(model_file='best_model.txt')

import json

with open('LGBM_Optuna_V_1_best_params_no_exta_features.json', 'w') as f:
    json.dump(best_params, f)

# import json
# from lightgbm import LGBMClassifier

# # Загрузка сохраненных параметров
# with open('best_params.json', 'r') as f:
#     loaded_params = json.load(f)

# # Обучение модели с загруженными параметрами
# model_with_best_params = LGBMClassifier(**loaded_params)
# model_with_best_params.fit(X_train, y_train)

# # Предсказания с новой моделью
# predictions = model_with_best_params.predict(some_new_data)


In [63]:
# Определение оптимального порога
precision, recall, thresholds = precision_recall_curve(y_test, test_preds)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_threshold = thresholds[f1_scores[:-1].argmax()]

print(f"Optimal Threshold: {optimal_threshold}")

Optimal Threshold: 0.2151395253285745


In [64]:
# Применение оптимального порога для классификации
y_pred_optimal = (test_preds > optimal_threshold).astype(int)

# Оценка качества модели с новым порогом на тестовой выборке
accuracy = accuracy_score(y_test, y_pred_optimal)
roc_auc = roc_auc_score(y_test, test_preds)
f1 = f1_score(y_test, y_pred_optimal)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC: {roc_auc}")
print(f"F1 Score: {f1}")
0.4726

Accuracy: 0.7464839523981248
ROC AUC: 0.7356069653347105
F1 Score: 0.47261815453863465
